In [ ]:
!pip install zemberek-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141211 sha256=b7eea7d51174168fbe8df79a07b2d5c1385bb0b13e124c7ab16898ce054b33ce
  Stored in directory: /root/.cache/pip/wheels/a7/20/bd/e1477d664f22d99989fd28ee1a43d6633dddb5cb9e801350d5
Successfully built antlr4-python3-runtime


In [ ]:
import re
import csv
import string

from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishMorphology,
)
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 27.391704320907593


2024-05-24 10:58:50,941 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 27.391704320907593



In [ ]:
import itertools

In [ ]:
import numpy as np
import pandas as pd
import random

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


2024-05-24 10:59:04,521 - numexpr.utils - INFO
Msg: NumExpr defaulting to 2 threads.



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
#drive bağlama
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#pozitif ve negatif txt dosyaları için bazı fonksiyonlar
def kelime_ekle(dosya_adı, kelime):
    try:
        with open(dosya_adı, 'r') as dosya:
            icerik = dosya.read()
            if kelime in icerik.split():
                print(f"'{kelime}' kelimesi zaten dosyada mevcut.")
                return
        with open(dosya_adı, 'a') as dosya:
            dosya.write('\n' + kelime)
        print(f"'{kelime}' kelimesi dosyaya başarıyla eklendi.")
    except FileNotFoundError:
        print("Dosya bulunamadı.")

def kelime_sil(dosya_adı, silinecek_kelime):
    try:
        with open(dosya_adı, 'r') as dosya:
            satırlar = dosya.readlines()

        with open(dosya_adı, 'w') as dosya:
            for satır in satırlar:
                if silinecek_kelime not in satır:
                    dosya.write(satır)

        print(f"'{silinecek_kelime}' kelimesi dosyadan başarıyla silindi.")
    except FileNotFoundError:
        print("Dosya bulunamadı.")


def kelime_arama(dosya_adı, aranan_kelime):
    try:
        with open(dosya_adı, 'r') as dosya:
            metin = dosya.read()
            kelimeler = metin.split()
            for kelime in kelimeler:
                if kelime == aranan_kelime:
                    return True
            return False
    except FileNotFoundError:
        print("Dosya bulunamadı.")

In [ ]:
# SENTENCE NORMALIZATION
def get_normalizasyon(example):
    return normalizer.normalize(example)

In [ ]:
#cumleler = "yrn"
#get_normalizasyon(cumleler)

In [ ]:
cumleler = """Ben bugün eve gelirken çok güzel bir kedi gördüm.
            Yrn okua gidicem
            Tmm, yarin havuza giricem ve aksama kadar yaticam :)
            ah aynen ya annemde fark ettı siz evinizden cıkmayın diyo
            gercek mı bu? Yuh! Artık unutulması bile beklenmiyo
            Hayır hayat telaşm olmasa alacam buraları gökdelen dikicem.
            yok hocam kesınlıkle oyle birşey yok
            herseyi soyle hayatında olmaması gerek bence boyle ınsanların falan baskı yapıyosa
            email adresim zemberek_python@loodos.com
            Kredi başvrusu yapmk istiyrum.
            Bankanizin hesp blgilerini ogrenmek istyorum.
            ii gzldi"""
get_normalizasyon(cumleler)

'ben bugün eve gelirken çok güzel bir kedi gördüm . yarın okula gideceğim tamam , yarın havuza gireceğim ve akşama kadar yatacağım :) ah aynen ya annemde fark etti siz evinizden çıkmayın diyor gerçek mi bu ? yuh ! artık unutulması bile beklenmiyor hayır hayat telaşı olmasa alacağım buraları gökdelen dikeceğim . yok hocam kesinlikle öyle bir şey yok herşeyi söyle hayatında olmaması gerek bence böyle insanların falan baskı yapıyorsa mail adresim zemberek_python@loodos.com kredi başvurusu yapmak istiyorum . bankanızın hesap bilgilerini öğrenmek istiyorum . ıı geldi'

In [ ]:
# def olumsuz_halini_bul(fiil):
#     olumsuz_ekler = ["miş","mış","amamak","ememek","mamak","memek","mak","mek","ama","eme","ma","me","mı","mi"]
#     olumsuz_hal = ["mek","mak","mamak","memek","mamak","memek","mak","mek","mamak","memek","mamak","memek","mamak","memek"]

#     for i in range(len(olumsuz_ekler)):
#         if olumsuz_ekler[i] in fiil:
#             return re.sub(olumsuz_ekler[i] + '.*$', olumsuz_hal[i], fiil)

#     analysis = morphology.analyze_sentence(fiil)
#     after = morphology.disambiguate(fiil, analysis)
#     for s in after.best_analysis():
#         kelime = s.item.lemma
#     return kelime

# fiiller = ["yapamayacağım", "sevmeyeceğim", "beğenemedim", "yapamadım","sevemedim","sevmek","sevmemek","sevmiyorum","yapamadım","yapmıyacağım","sevdim","beğen"]
# for fiil in fiiller:
#     print(olumsuz_halini_bul(fiil))

In [ ]:
def get_kelimeanaliz(example):
    gerekli_kelimeler = []
    indexleme = []

    analysis = morphology.analyze_sentence(example)
    after = morphology.disambiguate(example, analysis)
    for s in after.best_analysis():
        kelime = s.item.lemma
        obek = s.item.primary_pos.get_string_form()

        #if obek == 'Verb':
           #gerekli_kelimeler.append(s.surface_form() + " = " + obek)
           #indexleme.append(olumsuz_halini_bul(s.surface_form()))
           #indexleme.append(s.surface_form())

        if obek == 'Adj' or obek == 'Adv':
            gerekli_kelimeler.append(kelime + " = " + obek)
            indexleme.append(kelime)
    return indexleme

In [ ]:
cumle = "güzel bir ürün"
get_kelimeanaliz(cumle)

['güzel']

In [ ]:
def pozitif_mi(kelime):
    dosya_adı = '/content/drive/MyDrive/Sözlük/pozitif.txt'
    return kelime_arama(dosya_adı,kelime)

def negatif_mi(kelime):
    dosya_adı = '/content/drive/MyDrive/Sözlük/negatif.txt'
    return kelime_arama(dosya_adı,kelime)

In [ ]:
def save_to_csv(frekans_tablosu, dosya_adı):
    with open(dosya_adı, mode='w', newline='') as dosya:
        yazici = csv.writer(dosya)
        for item in frekans_tablosu:
            if item is not None:
                kelime, frekans = item
                yazici.writerow([kelime, frekans])

def load_from_csv(dosya_adı, tablo_boyutu):
    with open(dosya_adı, mode='r') as dosya:
        okuyucu = csv.reader(dosya)
        kelime_tablosu = [None] * tablo_boyutu
        for satir in okuyucu:
            kelime, frekans = satir
            indeks = hash_func(kelime, tablo_boyutu)
            step = 1
            while kelime_tablosu[indeks] is not None:
                indeks = linear_probe(indeks, step, tablo_boyutu)
                step += 1
            kelime_tablosu[indeks] = [kelime, int(frekans)]
    return kelime_tablosu

In [ ]:
def hash_func(kelime, boyut):
    return sum(map(ord, kelime)) % boyut

def linear_probe(index, step, boyut):
    return (index + step) % boyut

def kelime_cikar(metin):
    return metin.translate(str.maketrans("", "", string.punctuation + string.digits)).split()

def kelime_agirligi(metin_listesi, tablo_boyutu=5000):
    kelime_tablosu = [None] * tablo_boyutu
    kelime_tablosu = yuklenen_tablo

    for metin in metin_listesi:
        kelimeler = kelime_cikar(metin)

        for kelime in kelimeler:
            kelime = kelime.lower()


            if kelime.isnumeric():
                continue

            if pozitif_mi(kelime):
                ek = 1
            elif negatif_mi(kelime):
                ek = -1
            else:
                while True:
                    secim = input(f"'{kelime}' kelimesi pozitif mi (p) yoksa negatif mi (n)? (p/n): ")
                    if secim == 'p':
                        dosya_adı = '/content/drive/MyDrive/Sözlük/pozitif.txt'
                        kelime_ekle(dosya_adı,kelime)
                        ek = 1
                        break
                    elif secim == 'n':
                        dosya_adı = '/content/drive/MyDrive/Sözlük/negatif.txt'
                        kelime_ekle(dosya_adı,kelime)
                        ek = -1
                        break
                    else:
                        print("Geçersiz seçim. Lütfen 'p' veya 'n' girin.")

            indeks = hash_func(kelime, tablo_boyutu)
            step = 1

            while kelime_tablosu[indeks] is not None:
                if kelime_tablosu[indeks][0] == kelime:
                    kelime_tablosu[indeks][1] += ek
                    break
                indeks = linear_probe(indeks, step, tablo_boyutu)
                step += 1

            else:
                kelime_tablosu[indeks] = [kelime, ek]

    dosya_adı = "/content/drive/MyDrive/Sözlük/hash_table.csv"
    save_to_csv(kelime_tablosu, dosya_adı)
    return kelime_tablosu

def ara_tablo(kelime, frekans_tablosu, tablo_boyutu=5000):
    indeks = hash_func(kelime, tablo_boyutu)
    step = 1

    while frekans_tablosu[indeks] is not None:
        if frekans_tablosu[indeks][0] == kelime:
            #print(f"'{kelime}' kelimesi tabloda {frekans_tablosu[indeks][1]} kez geçiyor.")
            return frekans_tablosu[indeks][1]
        indeks = linear_probe(indeks, step, tablo_boyutu)
        step += 1

    #print(f"'{kelime}' kelimesi tabloda bulunamadı.")
    return 0

def print_frekans_tablosu(frekans_tablosu):
    for indeks, item in enumerate(frekans_tablosu):
        if item is not None:
            kelime, frekans = item
            print(f"Index {indeks}: {kelime}: {frekans}")

def en_yuksek_frekansli_kelime(frekans_tablosu):
    en_yuksek_frekans = 0
    en_yuksek_kelime = ""

    for item in frekans_tablosu:
        if item is not None:
            kelime, frekans = item
            if frekans > en_yuksek_frekans:
                en_yuksek_frekans = frekans
                en_yuksek_kelime = kelime

    return en_yuksek_frekans

def en_dusuk_frekansli_kelime(frekans_tablosu):
    en_dusuk_frekans = float('inf')  # Sonsuz gibi bir değerle başlatın
    en_dusuk_kelime = ""

    for item in frekans_tablosu:
        if item is not None:
            kelime, frekans = item
            if frekans < en_dusuk_frekans:
                en_dusuk_frekans = frekans
                en_dusuk_kelime = kelime

    return en_dusuk_frekans

def katsayı_hesapla(deger, min_deger, max_deger):
    if deger > 0:
        normalized_deger = (deger - 0) / (max_deger - 0)
        return normalized_deger
    elif deger < 0:
        normalized_deger = (deger - min_deger) / (0 - min_deger)
        return normalized_deger * 1 -1
    return 0

def normalize_frekans_tablosu(tablo):
    new_frekans_tablosu = tablo
    en_yuksek_puan = en_yuksek_frekansli_kelime(tablo)
    en_dusuk_puan = en_dusuk_frekansli_kelime(tablo)
    for indeks, item in enumerate(tablo):
        if item is not None:
            kelime, frekans = item
            new_frekans_tablosu[indeks][1]= katsayı_hesapla(tablo[indeks][1], en_dusuk_puan, en_yuksek_puan)
    return new_frekans_tablosu

In [ ]:
dosya_adı = "/content/drive/MyDrive/Sözlük/hash_table.csv"
yuklenen_tablo = load_from_csv(dosya_adı, 5000)

In [ ]:
dosya_adı = "/content/drive/MyDrive/Sözlük/hash_table.csv"
new_frekans_tablosu = load_from_csv(dosya_adı, 5000)
#print_frekans_tablosu(new_frekans_tablosu)
#katsayı_hesapla(9,-5,19)
katsayi = normalize_frekans_tablosu(new_frekans_tablosu)
print_frekans_tablosu(katsayi)

In [ ]:
ara_tablo("güzel",frekans_tablosu)

0

In [ ]:
#en_yuksek_frekansli_kelime(frekans_tablosu)

0

In [ ]:
#en_dusuk_frekansli_kelime(frekans_tablosu)

inf

In [ ]:
katsayı_hesapla(ara_tablo("güzel",frekans_tablosu),en_dusuk_frekansli_kelime(frekans_tablosu),en_yuksek_frekansli_kelime(frekans_tablosu))

0.425

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Sözlük/veri_seti.csv", encoding='utf-16')

In [ ]:
df.head()

,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz


In [ ]:
df.dropna(inplace=True)

In [ ]:
df["Durum"].value_counts()

Durum
Olumlu      4252
Olumsuz     4237
Tarafsız    2937
Name: count, dtype: int64

In [ ]:
df = df.loc[df["Durum"].isin(["Olumlu", "Olumsuz"])]

In [ ]:
df["Durum"].value_counts()

Durum
Olumlu     4252
Olumsuz    4237
Name: count, dtype: int64

In [ ]:
df.head()

,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz
6,kullanisli baya,Olumlu


In [ ]:
def etiketle(row):
    if row["Durum"] == "Olumlu":
        return 1
    return 0

In [ ]:
df["label"] = df.apply(lambda row: etiketle(row), axis=1)

In [ ]:
df.head()

,Görüş,Durum,label
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz,0
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu,1
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz,0
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz,0
6,kullanisli baya,Olumlu,1


In [ ]:
for _ in range(10):
    idx = random.randint(0, len(df)-1)
    print(df.iloc[idx]["Görüş"])
    print("-" * 40)

Ürün de sorun yok da Horoz Lojistik bugüne dek gördüğüm en berbat firmaydı. Hem Hepsiburada hem de İnceHesap firmalarının Horoz lojistikle çalışmaşaları hayal kırıklığı oldu. ABD&#8217;den aldığım ürün bir haftada elimdeydi bu ürün İstanbul&#8217;dan Ankara&#8217;ya 15 güne ancak gelebildi. O da ancak herkesi 10 kez aramam sayesinde oldu. Bu yüzden Horoz lojistikle çalışan firmalardan alışveriş yaparken bir kez daha düşünün derim. Resmen stres yaptılar...
----------------------------------------
ürün güzel hızlı sevkiyat
----------------------------------------
kapağı hic sağlam degil
----------------------------------------
Altus al 434 bulaşık mal inadını 4 yıl önce aldım. Garantisi bitti bir hafta sonra ana kartı yanmış dediler. 250 TL para verdim. 4 yılda 3. Kez ama kartı bozuldu. Bir daha altus mu tövbe
----------------------------------------
ben de pek etkili olduğunu söyleyemem. çok az etkili oldu . ama bir arkadaşımın önerisi üzerine kullandım. arkadaşımda 3 gün içerisinde yok

In [ ]:
x_deneme = df["Görüş"].to_list()
y_deneme = df["label"].to_list()

In [ ]:
#x
#y

In [ ]:
olumlu_df = df[df['Durum'] == 'Olumlu']

In [ ]:
#olumlu_df

In [ ]:
ilk_500_olumlu_df = olumlu_df.head(500)

In [ ]:
#ilk_500_olumlu_df

In [ ]:
olumsuz_df = df[df['Durum'] == 'Olumsuz']

In [ ]:
#olumsuz_df

In [ ]:
ilk_500_olumsuz_df = olumsuz_df.head(500)

In [ ]:
#ilk_500_olumsuz_df

In [ ]:
ilk_1000_df = pd.concat([ilk_500_olumlu_df, ilk_500_olumsuz_df])

In [ ]:
ilk_1000_df

,Görüş,Durum,label
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu,1
6,kullanisli baya,Olumlu,1
16,Mukemmel bir ürün kesinlikle alın,Olumlu,1
22,Arkadaşlar ürün çok güzel ve de sessiz çalışıy...,Olumlu,1
23,İndirimdeyken almıştık gayet kullanışlı güçlü ...,Olumlu,1
...,...,...,...
1283,İlan hatalı yayınlanan ilandaki cihaz resmi op...,Olumsuz,0
1285,Plastik yapısı çok kırılgan. Her an başına bir...,Olumsuz,0
1286,Ürün iyi fakat fişe taktığımda elektirik kaçır...,Olumsuz,0
1288,"Plastik , metal değil, orijinal değil, sadece ...",Olumsuz,0


In [ ]:
ilk_1000_df = ilk_1000_df["Görüş"].to_list()

In [ ]:
cleancumleler=[]
for cumle in ilk_1000_df:
  cleancumleler.append(get_normalizasyon(cumle))

In [ ]:
ilk_1000_df[:5]

['ses olayı süper....gece çalıştır sıkıntı yok.....kablo uzun işinizi çok rahat ettirir.....çekme olayı son derece güzel içiniz rahat olsun......diğerlerini saymıyorum bile.....',
 'kullanisli baya',
 'Mukemmel bir ürün kesinlikle alın',
 'Arkadaşlar ürün çok güzel ve de sessiz çalışıyo.Kargo hızlıydı fakat ben\nkargoyu takip edemedim(Ceva Lojistik).onun dışında bi sıkıntı yok.Mağaza\nda güvenilir bu arada',
 'İndirimdeyken almıştık gayet kullanışlı güçlü bir ürün']

In [ ]:
cleancumleler[:5]

['ses olayı süper ... . gece çalıştır sıkıntı yok ... . . kablo uzun işinizi çok rahat ettirir ... . . çekme olayı son derece güzel içiniz rahat olsun ... ... diğerlerini saymıyorum bile ... . .',
 'kullanışlı baya',
 'mükemmel bir ürün kesinlikle alın',
 'arkadaşlar ürün çok güzel ve de sessiz çalışıyor . kargo hızlıydı fakat ben kargoyu takip edemedim ( ceva lojistik ) . onun dışında bir sıkıntı yok . mağaza da güvenilir bu arada',
 'indirimdeyken almıştık gayet kullanışlı güçlü bir ürün']

In [ ]:
cleancumleler2=[]
for cumle in cleancumleler:
  cleancumleler2.append(get_kelimeanaliz(cumle))

In [ ]:
cleancumleler[:5]

['ses olayı süper ... . gece çalıştır sıkıntı yok ... . . kablo uzun işinizi çok rahat ettirir ... . . çekme olayı son derece güzel içiniz rahat olsun ... ... diğerlerini saymıyorum bile ... . .',
 'kullanışlı baya',
 'mükemmel bir ürün kesinlikle alın',
 'arkadaşlar ürün çok güzel ve de sessiz çalışıyor . kargo hızlıydı fakat ben kargoyu takip edemedim ( ceva lojistik ) . onun dışında bir sıkıntı yok . mağaza da güvenilir bu arada',
 'indirimdeyken almıştık gayet kullanışlı güçlü bir ürün']

In [ ]:
cleancumleler2[:5]

[['süper',
  'yok',
  'uzun',
  'çok',
  'rahat',
  'son',
  'güzel',
  'rahat',
  'diğer',
  'bile'],
 [],
 ['mükemmel', 'kesin'],
 ['çok', 'güzel', 'yok'],
 []]

In [ ]:
tam = cleancumleler
x = cleancumleler2
y = [1] * 500 + [0] * 500 #ilk 500 olumlu (1), son 500 olumsuz (0)

In [ ]:
x_train, x_test, y_train, y_test, tam_train, tam_test = train_test_split(x, y, tam, test_size=0.2)

In [ ]:
tam_train[:5]

['ürün oyuncak ütü gibi',
 'ürün maksimum iki kişilik bir aile için ideal , küçük bir buz dolabı . 1 +1 ev için ideal , bunları göz önünde bulundurarak almanızı tavsiye ederim . şimdilik sessiz çalışıyor , memnunum , kargo iki günde geldi , buz dolabını ben kurdum ( 5 dakika sürdü ) kendiniz kurabilirsiniz , servise hiç gerek yok , kutudan çıkarın fişe takın , hazır .',
 'ürün elime hala geçmediği için bilmiyorum',
 'ürün kullanışlı ve kaliteli tavsiye ederim . kullanıma göre fazlasıyla işimizi görüyor .',
 'almadan önce yorumları okudum , damlatıyor diye yazıldığını görünce tedirgin oldum ama yine de almaya karar verdim , kesinlikle damlatmıyor , belkide çok dolu olunca olabilir , bende hiç olmadı , harika görünümü var annem görünce bayıldı , günlük olarak kullanıyorum , kimi kapatma düğmesi yok demişti yanlış hatırlamıyorsam küçük yuvarlak düğme kapatma düğmesi , her halde bunu bilemediler şu ısıtıcımı kaldırdım , hem çaydanlık hem şu ışığı olarak kullanıyorum uzun zamandır böyle bir

In [ ]:
x_train[:5]

[[],
 ['maksimum', 'küçük', 'ideal', 'şimdilik', 'memnun', 'hiç', 'yok', 'hazır'],
 ['hâlâ'],
 ['fazlasıyla'],
 ['tedirgin',
  'yine',
  'kesin',
  'çok',
  'dolu',
  'hiç',
  'harika',
  'var',
  'yok',
  'küçük',
  'yuvarlak',
  'uzun',
  'böyle',
  'gerçekten',
  'harika',
  'sadece',
  'var']]

In [ ]:
y_train[:5]

[0, 1, 0, 1, 1]

In [ ]:
frekans_tablosu = kelime_agirligi(list(itertools.chain.from_iterable(x_train)))

In [ ]:
print_frekans_tablosu(frekans_tablosu)

Index 211: en: 84
Index 212: ne: 74
Index 213: ta: 4
Index 219: az: -20
Index 305: cam: 1
Index 311: dar: -4
Index 312: dik: -1
Index 314: hem: 2
Index 317: bol: 2
Index 318: hep: 15
Index 320: ilk: 68
Index 321: pek: 22
Index 322: tam: 52
Index 323: bor: 2
Index 324: tek: 50
Index 325: iri: -2
Index 327: ise: 14
Index 328: gri: 1
Index 329: var: 113
Index 331: iyi: 322
Index 332: pis: -3
Index 334: mor: 2
Index 335: sol: 2
Index 336: son: 14
Index 337: boz: -1
Index 339: yok: -238
Index 343: tiz: 3
Index 347: zor: -41
Index 349: toz: 1
Index 398: daha: 254
Index 399: kaba: -2
Index 407: baki: 2
Index 408: feci: 1
Index 409: acil: -1
Index 410: fena: 8
Index 412: bile: -74
Index 413: sade: 4
Index 415: ince: 15
Index 417: asla: -11
Index 418: onca: 3
Index 419: mail: 5
Index 423: geri: 51
Index 424: ufak: -11
Index 425: peki: 2
Index 426: emin: 1
Index 427: keza: 2
Index 428: eski: -27
Index 429: mini: 3
Index 430: mavi: 7
Index 431: krem: 2
Index 433: elim: 2
Index 434: eksi: -1
Index

In [ ]:
x_test

[['uzun', 'ayrıca', 'sadece', 'hiç', 'geri'],
 ['dolayısıyla', 'çok', 'zayıf', 'sadece', 'toz', 'memnun'],
 ['süper'],
 ['olumlu', 'gayet', 'güzel', 'en', 'güzel'],
 ['nasıl',
  'hiç',
  'doğal',
  'yok',
  'özellikle',
  'çok',
  'meselâ',
  'resmen',
  'beyaz',
  'çok',
  'pişman',
  'iyi',
  'pişman',
  'daha',
  'önce'],
 ['böyle', 'yanlış', 'nasıl', 'ilgili'],
 [],
 ['muhteşem'],
 ['güzel'],
 ['yok', 'geç', 'büyük'],
 ['pişman', 'açıkçası', 'en', 'büyük', 'tam', 'aynı', 'var', 'hiç', 'memnun'],
 [],
 ['çok', 'ufak', 'var', 'belli', 'belirsiz', 'genel', 'hoş', 'yok'],
 ['gayet', 'hafif', 'yavaş', 'acil'],
 ['olumlu'],
 ['yok', 'yerine', 'mini', 'yok', 'minnacık'],
 ['çok', 'hafif', 'gayet', 'iyi', 'kolay', 'ince', 'var'],
 ['yok', 'soğuk', 'maalesef', 'yeterli'],
 ['siyah', 'siyah', 'beyaz', 'belli', 'hiç', 'yok', 'kısacası', 'ne'],
 ['zaten', 'mağdur', 'altüst'],
 ['hâlâ', 'çok'],
 ['sadece', 'en', 'fazla'],
 [],
 ['çok', 'güzel'],
 ['güzel'],
 ['büyük', 'iyi', 'iyi', 'beyaz', 'be

In [ ]:
def yorum_degerlendir_detayli(liste, frekans_tablosu, tablo_boyutu=5000):
    sonuc = []
    toplam_agirlik = 0
    for cumle in liste:
        toplam_agirlik = 0
        cumle_sonuc = []
        for kelime in cumle:
            katsayi = katsayı_hesapla(ara_tablo(kelime,frekans_tablosu,5000),en_dusuk_frekansli_kelime(frekans_tablosu),en_yuksek_frekansli_kelime(frekans_tablosu))
            cumle_sonuc.append((kelime,katsayi))
        sonuc.append(cumle_sonuc)
    return sonuc

In [ ]:
def yorum_sonuc(cleancumleler2, frekans_tablosu, tablo_boyutu=5000):
    toplam_agirliklar = []
    for kelimeler in cleancumleler2:
      toplam_agirlik = 0
      for kelime in kelimeler:
        toplam_agirlik += katsayı_hesapla(ara_tablo(kelime,frekans_tablosu,5000),en_dusuk_frekansli_kelime(frekans_tablosu),en_yuksek_frekansli_kelime(frekans_tablosu))
      toplam_agirliklar.append(toplam_agirlik)
    return toplam_agirliklar

In [ ]:
#yorum_degerlendir_detayli(x_test, frekans_tablosu)

In [ ]:
#yorum_degerlendir_detayli(x_test, frekans_tablosu)

In [ ]:
cumle = ["güzel bir elbise aldım","kötü bir elbise"]

In [ ]:
normal_cumle=get_normalizasyon(cumle)

In [ ]:
kelimeler = get_kelimeanaliz(normal_cumle)

In [ ]:
tablo = kelime_agirligi(kelimeler)

In [ ]:
print_frekans_tablosu(tablo)

In [ ]:
yorum_degerlendir_detayli(x_test, frekans_tablosu)[10]

[('pişman', -0.2016806722689075),
 ('açıkçası', 0.009186351706036745),
 ('en', 0.11023622047244094),
 ('büyük', 0.06430446194225722),
 ('tam', 0.06824146981627296),
 ('aynı', 0.07217847769028872),
 ('var', 0.14829396325459318),
 ('hiç', -0.7478991596638656),
 ('memnun', 0.21916010498687663)]

In [ ]:
yorum_sonuc(x_test, frekans_tablosu)[10]

-0.2579787820640067

In [ ]:
x_testnew=[]
for a in x_test:
    x_testnew.append(", ".join(a))

In [ ]:
dfnew = pd.DataFrame()

dfnew["Görüş"]=pd.DataFrame(tam_test)
dfnew["label"]=pd.DataFrame(y_test)
dfnew["Kelimeler"]=pd.DataFrame(x_testnew)
dfnew["Toplam Ağırlık"]=pd.DataFrame(yorum_sonuc(x_test, frekans_tablosu))
dfnew['binary_sonuçlar'] = np.where(dfnew['Toplam Ağırlık'] > 0, 1,
                                    np.where(dfnew['Toplam Ağırlık'] == 0, -1, 0))
dfnew['Doğruluk'] = dfnew['label'] == dfnew['binary_sonuçlar']

In [ ]:
dfnew

,Görüş,label,Kelimeler,Toplam Ağırlık,binary_sonuçlar,Doğruluk
0,el süpürgesinin içinden çıkan uzun aparat kolt...,0,"uzun, ayrıca, sadece, hiç, geri",-0.827047,0,True
1,ürünün motor gücü ( dolayısıyla emiş gücü ) ço...,0,"dolayısıyla, çok, zayıf, sadece, toz, memnun",0.903240,1,False
2,süper bir alışveriş oldu .,1,süper,0.043307,1,True
3,"olumlu yönleri : gayet kaliteli ve güzel , bu ...",1,"olumlu, gayet, güzel, en, güzel",1.299213,1,True
4,bende burdaki yorumları okuyarak bu tvyi aldım...,0,"nasıl, hiç, doğal, yok, özellikle, çok, meselâ...",0.793621,1,False
...,...,...,...,...,...,...
195,bıçak ilk günden kötü oldu ! nasıl ürün bu,0,"ilk, kötü, nasıl",-0.161836,0,True
196,ürün satılırken belirtilen ses çıkış için blue...,0,"genel, mobil, yok, mobil, yok, genel, genel, i...",-1.878769,0,True
197,kargo berbat ayın 9 unda kargoya teslim edilmi...,0,berbat,-0.151261,0,True
198,kesinlikle rezalet bu kadar rezillik olmaz . i...,0,"kesinlikle, rezil, belli, kısmî, dolu, yoğun, ...",0.246179,1,False


In [ ]:
true_count = dfnew['Doğruluk'].sum()
total_count = len(dfnew)
true_ratio = (true_count / total_count) * 100
hicbir_sonuc_olmayan = (dfnew['binary_sonuçlar'] == -1).sum()
print("Cümle sayısı:", total_count)
print("Doğruluk oranı: {:.2f}%".format(true_ratio))
print("----------------------------------------------")
print("Hiçbir sonuc vermeyen cümle sayısı:",hicbir_sonuc_olmayan)
total_count -= hicbir_sonuc_olmayan
true_ratio = (true_count / total_count) * 100
print("Sonuc vermeyen cümleler çıkarılınca Doğruluk oran: {:.2f}%".format(true_ratio))


Cümle sayısı: 200
True değerlerin oranı: 55.50%
----------------------------------------------
Hiçbir sonuc vermeyen cümle sayısı: 25
Sonuc vermeyen cümleler çıkarılınca oran: 63.43%


In [ ]:
def cumle_analiz(cumle):
  normal_cumle = get_normalizasyon(cumle)
  kelimeler = get_kelimeanaliz(normal_cumle)
  kelime_agirligi(kelimeler)
  kelimeler=[kelimeler]
  yorum = yorum_degerlendir_detayli(kelimeler, frekans_tablosu)
  cumle_deger = yorum_sonuc(kelimeler, frekans_tablosu)
  print("Yorumun: ", cumle)
  print("-"*50)
  print("Yorumun düzenlenmiş hali: ", normal_cumle)
  print("-"*50)
  print("Yorumdan çıkarılan kelimeler: ",kelimeler)
  print("-"*50)
  print("Kelimelerin değerleri: ",yorum)
  print("-"*50)
  print("Yorumun genel değeri: ",cumle_deger)

In [ ]:
cumle = "Zamanla sıkıcı bir hal aldı. Sadece hikayeyi karışık hale getirdiler. Karıştıkça sıkıcı oldu."

In [ ]:
cumle_analiz(cumle)

Yorumun:  Zamanla sıkıcı bir hal aldı. Sadece hikayeyi karışık hale getirdiler. Karıştıkça sıkıcı oldu.
--------------------------------------------------
Yorumun düzenlenmiş hali:  zamanla sıkıcı bir hal aldı . sadece hikayeyi karışık hale getirdiler . karıştıkça sıkıcı oldu .
--------------------------------------------------
Yorumdan çıkarılan kelimeler:  [['sıkıcı', 'sadece', 'karışık', 'sıkıcı']]
--------------------------------------------------
Kelimelerin değerleri:  [[('sıkıcı', -0.01680672268907568), ('sadece', -0.2857142857142857), ('karışık', -0.008403361344537785), ('sıkıcı', -0.01680672268907568)]]
--------------------------------------------------
Yorumun genel değeri:  [-0.32773109243697485]
